In [1]:
### Cu 003 processing ###


#%% load the packages
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

import defdap.hrdic as hrdic
import defdap.ebsd as ebsd
import defdap.experiment as experiment

from pathlib import Path

import copy 
import pandas as pd
from datetime import datetime

# get dictools stuff 
import sys
sys.path.append("c:/work/hrdic-tools/")
import dictools

plt.rcParams['svg.fonttype'] = 'none'

%matplotlib qt

Loading ITKPyBase... Loading ITKPyBase... Loading ITKCommon... Loading ITKCommon... Loading ITKStatistics... Loading ITKStatistics... Loading ITKImageFilterBase... Loading ITKImageFilterBase... Loading ITKTransform... Loading ITKTransform... Loading ITKImageFrequency... Loading ITKImageFrequency... Loading ITKIOImageBase... Loading ITKIOBMP... Loading ITKIOBMP... Loading ITKIOBioRad... Loading ITKIOBioRad... Loading ITKIOBruker... Loading ITKIOBruker... Loading ITKIOGDCM... Loading ITKIOGDCM... Loading ITKIOIPL... Loading ITKIOIPL... Loading ITKIOGE... Loading ITKIOGE... Loading ITKIOGIPL... Loading ITKIOGIPL... Loading ITKIOHDF5... Loading ITKIOHDF5... Loading ITKIOJPEG... Loading ITKIOJPEG... Loading ITKIOJPEG2000... Loading ITKIOJPEG2000... Loading ITKIOTIFF... Loading ITKIOTIFF... Loading ITKIOLSM... Loading ITKIOLSM... Loading ITKIOMINC... Loading ITKIOMINC... Loading ITKIOMRC... Loading ITKIOMRC... Loading ITKIOMeta... Loading ITKIOMeta... Loading ITKIONIFTI... Loading ITKIONIFTI

<itkTemplate itk::TileConfiguration>
Options:
  [2,]
  [3,]
  [4,]


In [2]:
# read rig file and calculate some stress and strain measures 
rootfile = './ofhc_cu_5pc'
rigfile = rootfile + '.csv'
logfile = rootfile + '_log.txt'


rigdat = pd.read_csv(rigfile)

g_length = 12   # mm
g_width  = 1.5  # mm
g_thick  = 1.1035 # mm

# correct for unit slashes 
rigdat = rigdat.rename(columns={'Elapsed Time (s)': 'Elapsed Time / s',
                                'LVDT Position (µm)':'LVDT Position / µm',
                                'Elongation (µm)':'Elongation / µm',
                                'Force (N)' : 'Force / N',
                                'Speed (µm/s)': 'Speed / µm/s',
                                'Sample Temperature (°C)':'Sample Temperature / °C'
                                })



rigdat['Time'] = pd.to_datetime(rigdat['Time'])
rigdat['Eng Stress / MPa']  = rigdat['Force / N']/(g_width*g_thick)
rigdat['Eng Strain']        = rigdat['Elongation / µm']/(g_length*1000) 
rigdat['True Stress / MPa'] = rigdat['Eng Stress / MPa']*(1 + rigdat['Eng Strain'])
rigdat['True Strain']       = np.log(1 + rigdat['Eng Strain'])


# read log file
riglog = pd.read_fwf(logfile,header=None,usecols=[1,2])

riglog[1] = pd.to_datetime(riglog[1])

riglog = riglog.rename(columns={1:'time',2:'log'})

# find times when images were taken
mask = riglog['log'].str.contains('Set acquisition detector settings...')
imstarts = riglog[mask]

mask = riglog['log'].str.contains('Step')
steps = riglog[mask]

df = pd.concat([imstarts,steps])
df = df.sort_index()

# check for duplicates
s = df['log'].shift()

mask = df['log'].eq(s)

# remove duplicates - needed for the removal of multiple ROIs in one step 
df = df[~mask]

# pick out times when image acquisition is started 
mask = df['log'].str.contains('Set acquisition detector settings...')
image_start_times = df['time'][mask]

# new dataframe to store data from the start of each imaging run 
rigdatimage = pd.DataFrame()

for i,t in enumerate(image_start_times):
    
    # look through rigdat dataframe for minimum time difference between given timestamp and image_start_times 
    dt = abs(rigdat['Time'] - t)

    thistimeidx = dt.idxmin()

    rigdatimage = pd.concat([rigdatimage, rigdat.loc[thistimeidx]],axis=1)

rigdatimage = rigdatimage.transpose()

In [3]:
x = 'Eng Strain' 
y = 'Eng Stress / MPa'



fig,ax = plt.subplots()
fig.set_size_inches(12,8)
plt.rcParams['svg.fonttype'] = 'none'


ax.plot(rigdat[x],rigdat[y],'k',linewidth = 3)
ax.plot(rigdatimage[x],rigdatimage[y],'.',markerfacecolor='orange',markeredgecolor='green',markersize=20,alpha=0.9)

ax.set_ylabel(y)
ax.set_xlabel(x)

plt.savefig('stress_strain.svg')

In [4]:
x = 'Elapsed Time / s'
y = 'Elongation / µm'

fig,ax = plt.subplots(2,1,sharex=True)

ax[0].plot(rigdat[x],rigdat[y],'k',linewidth = 1)
ax[0].plot(rigdatimage[x],rigdatimage[y],'.',markerfacecolor='orange',markeredgecolor='green',markersize=15,alpha=0.9)

ax[0].set_ylabel(y)

x = 'Elapsed Time / s'
y = 'Eng Stress / MPa'

ax[1].plot(rigdat[x],rigdat[y],'k',linewidth = 1)
ax[1].plot(rigdatimage[x],rigdatimage[y],'.',markerfacecolor='orange',markeredgecolor='green',markersize=15,alpha=0.9)

ax[1].set_xlabel(x)
ax[1].set_ylabel(y)

def seconds2days(t):
    return t * (1/60) * (1/60) * (1/24) 

def days2seconds(t):
    return t * 60 * 60

def seconds2hours(t):
    return t * (1/60) * (1/60)

def hours2seconds(t):
    return t * 60 * 60

secax = ax[0].secondary_xaxis('top',functions=(seconds2hours,hours2seconds))
secax.set_xlabel('Elapsed Time / h')

fig.set_size_inches(7,7)

plt.tight_layout()

# plt.savefig('Image_capture_points_elong_stress_time.tif',dpi=500)

In [5]:
x = 'Elapsed Time / s'
y = 'Elongation / µm'

fig,ax = plt.subplots(2,1,sharex=False)

ax[0].plot(rigdat[x]/(60*60),rigdat[y],'k',linewidth = 2)
ax[0].plot(rigdatimage[x]/(60*60),rigdatimage[y],'.',markerfacecolor='orange',markeredgecolor='green',markersize=30,alpha=0.9)

ax[0].set_xlabel('Elapsed Time / h')
ax[0].set_ylabel(y)

x = 'Eng Strain'
y = 'Eng Stress / MPa'

ax[1].plot(rigdat[x],rigdat[y],'k',linewidth = 2)
ax[1].plot(rigdatimage[x],rigdatimage[y],'.',markerfacecolor='orange',markeredgecolor='green',markersize=30,alpha=0.9)

ax[1].set_xlabel(x)
ax[1].set_ylabel(y)

def seconds2days(t):
    return t * (1/60) * (1/60) * (1/24) 

def days2seconds(t):
    return t * 60 * 60

def seconds2hours(t):
    return t * (1/60) * (1/60)

def hours2seconds(t):
    return t * 60 * 60

# secax = ax[0].secondary_xaxis('top',functions=(seconds2hours,hours2seconds))
# secax.set_xlabel('Elapsed Time / h')

fig.set_size_inches(12,16)

plt.tight_layout()

plt.savefig('EXPO_stress_strain.svg')

In [3]:
exp = experiment.Experiment()

dic_dir = Path('./DIC/')
ebsd_pre_dir = Path('./Pre_EBSD/')
ebsd_post_dir = Path('./Post_EBSD/')

dic_frame = experiment.Frame()
for dic_file in sorted(dic_dir.glob('map_*.txt')):
    hrdic.Map(dic_file, experiment=exp, frame=dic_frame,data_type='openpiv')
    
hfw = 20 # microns
pixelwidth = 2048
pixelsize = 20/2048

for inc, dic_map in exp.iter_over_maps('hrdic'):
    dic_map.set_scale(pixelsize)
    dic_map.set_crop(left=100,right=100,top=100,bottom=100)
    # dic_map.plot_map('max_shear',vmin=0,vmax=0.01,plot_scale_bar=True)
    print(dic_map)
    
ebsd_pre_frame = experiment.Frame()
ebsd.Map(ebsd_pre_dir / 'map.cpr',increment=exp.increments[0], frame=ebsd_pre_frame)


# dic_map = exp.increments[-1].maps['hrdic']
# dic_map.set_homog_point(vmin=0,vmax=0.2)

# ebsd_map = exp.increments[0].maps['ebsd']
# ebsd_map.set_homog_point()

ebsd_map = exp.increments[0].maps['ebsd']
# dic_frame.homog_points = [(952, 915),
#  (1506, 676),
#  (1597, 159),
#  (858, 257),
#  (1143, 790),
#  (1205, 281),
#  (1769, 643),
#  (1273, 1177),
#  (1539, 1261),
#  (1761, 1756),
#  (1109, 1471),
#  (1565, 1708),
#  (82, 448),
#  (304, 653),
#  (635, 761),
#  (923, 628),
#  (927, 1206),
#  (235, 1392),
#  (871, 1536),
#  (96, 1833)]

dic_frame.homog_points = [(952, 825),
 (1506, 586),
 (1597, 69),
 (858, 167),
 (1143, 700),
 (1205, 191),
 (1769, 553),
 (1273, 1097),
 (1539, 1171),
 (1761, 1666),
 (1109, 1381),
 (1565, 1618),
 (82, 358),
 (304, 563),
 (635, 671),
 (923, 538),
 (927, 1116),
 (235, 1302),
 (871, 1446),
 (96, 1743)]

ebsd_map = exp.increments[0].maps['ebsd']
ebsd_pre_frame.homog_points = [(1946, 1565),
 (2375, 1384),
 (2443, 999),
 (1877, 1079),
 (2094, 1469),
 (2142, 1091),
 (2574, 1359),
 (2193, 1760),
 (2396, 1823),
 (2571, 2196),
 (2065, 1989),
 (2418, 2156),
 (1279, 1218),
 (1454, 1369),
 (1700, 1457),
 (1925, 1345),
 (1929, 1788),
 (1391, 1934),
 (1881, 2037),
 (1281, 2274)]

 # plot maps
for inc, dic_map in exp.iter_over_maps('hrdic'):
    dic_map.set_crop
    dic_map.link_ebsd_map(ebsd_map, transform_type="polynomial",order=2)
    # dic_map.plot_map(
    #     'max_shear', vmin=0, vmax=0.10, 
    #     plot_scale_bar=False, plot_gbs='line'
    # )
    plt.tight_layout()

Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded OpenPIV n/a data (dimensions: 1999 x 1999 pixels, sub-window size: 16 x 16 pixels)
Loaded Ope

In [3]:
dic_map = exp.increments[-1].maps['hrdic']
dilation = 5

# generate mask to apply to all maps to extract grain boundaries and grain cores 
# returns arrays of separated strain components such that first slice is cores and 

def mask_and_extract(dic_map,mask):
    # masks DIC map data BUT not in the DIC map object
    # extracts the data first, then masks, then returns the data 

    # max shear
    ems = copy.deepcopy(dic_map.data['max_shear'])
    ems[mask] = np.nan

    # strain components 
    e = copy.deepcopy(dic_map.data['e'])
    e[:,:,mask] = np.nan

    # displacements 
    d = copy.deepcopy(dic_map.data['displacement'])
    d[:,mask] = np.nan

    return ems, e, d


def cores_vs_boundaries_histograms(dic_map,dilation,plotMasks = True,plotHists = True):
    # splits map into grain boundary and grain core regions 
    # plots histograms and spits out some stats 

    # generate mask
    dic_map.generate_threshold_mask(dic_map.data['grain_boundaries'].image,dilation=dilation)

    # closes defdap builtin plots 
    plt.close()
    plt.close()

    # core mask and inverse mask for boundaries
    gcmask = dic_map.mask
    gbmask = ~gcmask

    # extract pair of boundary and core zones 
    emsgc,egc,dgc = mask_and_extract(dic_map,gcmask)
    emsgb,egb,dgb = mask_and_extract(dic_map,gbmask)
    
    # plot the two masked regions 
    if plotMasks == True:
        fig,ax = plt.subplots(1,2)

        ax[0].set_title('Grain cores')
        ax[0].imshow(emsgc,vmin=0,vmax=0.1)
        ax[1].set_title('Grain boundaries')
        ax[1].imshow(emsgb,vmin=0,vmax=0.1)

        plt.tight_layout()

    
    # histogram comparison 
    if plotHists == True: 
        fig,ax = plt.subplots(2,2)

        # e11
        bins = 50
        range = (-0.1,0.1)
        ax[0,0].hist(egc[0,0,:,:].flatten(),bins=bins,range=range,density=True,alpha=0.5)
        ax[0,0].hist(egb[0,0,:,:].flatten(),bins=bins,range=range,density=True,alpha=0.5)
        ax[0,0].set_xlabel('E11')
        ax[0,0].set_ylabel('p. d.')

        # e22
        bins = 50
        range = (-0.1,0.1)
        ax[1,1].hist(egc[1,1,:,:].flatten(),bins=bins,range=range,density=True,alpha=0.5)
        ax[1,1].hist(egb[1,1,:,:].flatten(),bins=bins,range=range,density=True,alpha=0.5)
        ax[1,1].set_xlabel('E22')
        ax[1,1].set_ylabel('p. d.')

        # e12
        bins = 50
        range = (-0.1,0.1)
        ax[0,1].hist(egc[0,1,:,:].flatten(),bins=bins,range=range,density=True,alpha=0.5)
        ax[0,1].hist(egb[0,1,:,:].flatten(),bins=bins,range=range,density=True,alpha=0.5)
        ax[0,1].set_xlabel('E12')
        ax[0,1].set_ylabel('p. d.')

        # max shear
        bins = 50
        range = (0,0.1)
        ax[1,0].hist(emsgc.flatten(),bins=bins,range=range,density=True,alpha=0.5)
        ax[1,0].hist(emsgb.flatten(),bins=bins,range=range,density=True,alpha=0.5)
        ax[1,0].set_xlabel('Ems')
        ax[1,0].set_ylabel('p. d.')

        plt.tight_layout()

            
    # stats table
    print('Grain core vs grain boundary')
    print('===========================================================')
    print('E11 ')
    print('Grain core:       ' + str(round( np.nanmedian(egc[0,0,:,:].flatten()),5 )) + ' +/- ' +str(round( np.nanstd(egc[0,0,:,:].flatten()),5 )) )
    print('Grain boundary:   ' + str(round( np.nanmedian(egb[0,0,:,:].flatten()),5 )) + ' +/- ' +str(round( np.nanstd(egb[0,0,:,:].flatten()),5 )) )
    print('===========================================================')
    print('E22 ')
    print('Grain core:       ' + str(round( np.nanmedian(egc[1,1,:,:].flatten()),5 )) + ' +/- ' +str(round( np.nanstd(egc[1,1,:,:].flatten()),5 )) )
    print('Grain boundary:   ' + str(round( np.nanmedian(egb[1,1,:,:].flatten()),5 )) + ' +/- ' +str(round( np.nanstd(egb[1,1,:,:].flatten()),5 )) )
    print('===========================================================')
    print('E12 ')
    print('Grain core:       ' + str(round( np.nanmedian(egc[0,1,:,:].flatten()),5 )) + ' +/- ' +str(round( np.nanstd(egc[0,1,:,:].flatten()),5 )) )
    print('Grain boundary:   ' + str(round( np.nanmedian(egb[0,1,:,:].flatten()),5 )) + ' +/- ' +str(round( np.nanstd(egb[0,1,:,:].flatten()),5 )) )
    print('===========================================================')
    print('E max shear ')
    print('Grain core:       ' + str(round( np.nanmedian(emsgc.flatten()),5 )) + ' +/- ' +str(round( np.nanstd(emsgc.flatten()),5 )) )
    print('Grain boundary:   ' + str(round( np.nanmedian(emsgb.flatten()),5 )) + ' +/- ' +str(round( np.nanstd(emsgb.flatten()),5 )) )
    print('===========================================================')


    # package things up
    ems = np.stack([emsgc,emsgb])
    e11 = np.stack([egc[0,0,:,:],egb[0,0,:,:]])
    e12 = np.stack([egc[0,1,:,:],egb[0,1,:,:]])
    e22 = np.stack([egc[1,1,:,:],egb[1,1,:,:]])

    return ems,e11,e12,e22

In [4]:
emsgc = []
emsgb = []

e11gc = []
e11gb = []

e12gc = []
e12gb = []

e22gc = []
e22gb = []

for inc, dic_map in exp.iter_over_maps('hrdic'):
    ems_s,e11_s,e12_s,e22_s = cores_vs_boundaries_histograms(dic_map,dilation,plotMasks = False, plotHists = False)
    emsgc.append(ems_s[0,:,:].flatten())
    emsgb.append(ems_s[1,:,:].flatten())
    
    e11gc.append(e11_s[0,:,:].flatten())
    e11gb.append(e11_s[1,:,:].flatten())

    e12gc.append(e12_s[0,:,:].flatten())
    e12gb.append(e12_s[1,:,:].flatten())

    e22gc.append(e22_s[0,:,:].flatten())
    e22gb.append(e22_s[1,:,:].flatten())

Finished building quaternion array (0:00:46) 
Finished finding grain boundaries (0:02:00) 
Filtering will remove 991717 \ 3996001 (24.818 %) datapoints in map
Filtering will remove 897184 \ 3236401 (27.722 %) datapoints in cropped map
Use apply_threshold_mask function to apply this filtering to data
Grain core vs grain boundary
E11 
Grain core:       0.00023 +/- 0.00128
Grain boundary:   0.00022 +/- 0.00128
E22 
Grain core:       3e-05 +/- 0.00158
Grain boundary:   4e-05 +/- 0.0016
E12 
Grain core:       2e-05 +/- 0.00096
Grain boundary:   2e-05 +/- 0.00096
E max shear 
Grain core:       0.00114 +/- 0.00067
Grain boundary:   0.00114 +/- 0.00068
Filtering will remove 991717 \ 3996001 (24.818 %) datapoints in map
Filtering will remove 897184 \ 3236401 (27.722 %) datapoints in cropped map
Use apply_threshold_mask function to apply this filtering to data
Grain core vs grain boundary
E11 
Grain core:       0.00088 +/- 0.00143
Grain boundary:   0.00089 +/- 0.00144
E22 
Grain core:       5e-0

In [34]:
plt.imshow(ems_s[1,:,:],vmin=0,vmax=0.1)

Figures for EXPO

In [79]:
fig,ax = plt.subplots()

fig.set_size_inches(20,20)
ebsd_map = exp.increments[0].maps['ebsd']
ebsd_map.plot_ipf_map([1,0,0],plot_scale_bar=True,fig=fig,ax=ax)


In [80]:
plt.savefig('EBSD_small.tif',dpi=200)

In [76]:
fig,ax = plt.subplots()

fig.set_size_inches(20,20)
plt.imshow(ems_s[1,:,:],vmin=0,vmax=0.1)
plt.tight_layout()
plt.savefig('gbs_mask.tif',dpi=100)
# plt.savefig('EBSD.tif',dpi=200)

NameError: name 'ems_s' is not defined

In [40]:
fig,ax = plt.subplots()

fig.set_size_inches(20,20)
plt.imshow(ems_s[0,:,:],vmin=0,vmax=0.1)
plt.tight_layout()
plt.savefig('gcs_mask.tif',dpi=100)

In [72]:
fig,ax = plt.subplots()

cmap = plt.get_cmap('viridis').copy()
cmap.set_under('black')
cmap.set_over('white')


fig.set_size_inches(20,20)
dic_map = exp.increments[5].maps['hrdic']
dic_map.plot_map('max_shear',vmin=0.002,vmax=0.01,plot_scale_bar=True,cmap=cmap,fig=fig,ax=ax)
plt.tight_layout()


In [73]:
plt.savefig('strain_map_step6.tif',dpi=200)

In [101]:
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams.update({'font.size': 26})
plt.rc('xtick', labelsize=24) 
plt.rc('ytick', labelsize=24) 

fig,ax = plt.subplots(1,4)

fig.set_size_inches(30,7.5)

# fig,ax = plt.subplots(len(d1),1,sharex=True)

# for i in range(0,len(d1)):
#     ax[i].hist(d1[i],bins=100,range=(0,0.1),density=True,alpha=0.5)
#     ax[i].hist(d2[i],bins=100,range=(0,0.1),density=True,alpha=0.5)


# calculate global E11 values 
e11glob = []

for inc, dic_map in exp.iter_over_maps('hrdic'):
    e11glob.append(np.nanmean(dic_map.data['e'][0,0].flatten()))

e11glob = e11glob[:-1]

ccol    = 'forestgreen'
cmark   = 'h'

bcol    = 'darkorange'
bmark   = 'x'

tcol    = 'black'
tmark   = '+'

ms = 15

# e11
d1 = e11gb[:-1]
d2 = e11gc[:-1]
d1mean = [np.nanmean(x) for x in d1]
d2mean = [np.nanmean(x) for x in d2]

d3ar = np.hstack((np.asarray(d1),np.asarray(d2)))
d3mean = np.nanmean(d3ar,1)


ax[0].plot(e11glob,d1mean,marker=bmark,color=bcol,linestyle='None',markersize=ms)
ax[0].plot(e11glob,d2mean,marker=cmark,color=ccol,linestyle='None',markersize=ms)
ax[0].plot(e11glob,d3mean,marker=tmark,color=tcol,linestyle='None',markersize=ms)


# e22 
d1 = e22gb[:-1]
d2 = e22gc[:-1]
d1mean = [np.nanmean(x) for x in d1]
d2mean = [np.nanmean(x) for x in d2]

d3ar = np.hstack((np.asarray(d1),np.asarray(d2)))
d3mean = np.nanmean(d3ar,1)

ax[1].plot(e11glob,d1mean,marker=bmark,color=bcol,linestyle='None',markersize=ms)
ax[1].plot(e11glob,d2mean,marker=cmark,color=ccol,linestyle='None',markersize=ms)
ax[1].plot(e11glob,d3mean,marker=tmark,color=tcol,linestyle='None',markersize=ms)


# e12 
d1 = e12gb[:-1]
d2 = e12gc[:-1]
d1mean = [np.nanmean(x) for x in d1]
d2mean = [np.nanmean(x) for x in d2]

d3ar = np.hstack((np.asarray(d1),np.asarray(d2)))
d3mean = np.nanmean(d3ar,1)

ax[2].plot(e11glob,d1mean,marker=bmark,color=bcol,linestyle='None',markersize=ms)
ax[2].plot(e11glob,d2mean,marker=cmark,color=ccol,linestyle='None',markersize=ms)
ax[2].plot(e11glob,d3mean,marker=tmark,color=tcol,linestyle='None',markersize=ms)


# ems 
d1 = emsgb[:-1]
d2 = emsgc[:-1]
d1mean = [np.nanmean(x) for x in d1]
d2mean = [np.nanmean(x) for x in d2]

d3ar = np.hstack((np.asarray(d1),np.asarray(d2)))
d3mean = np.nanmean(d3ar,1)

ax[3].plot(e11glob,d1mean,marker=bmark,color=bcol,linestyle='None',markersize=ms)
ax[3].plot(e11glob,d2mean,marker=cmark,color=ccol,linestyle='None',markersize=ms)
ax[3].plot(e11glob,d3mean,marker=tmark,color=tcol,linestyle='None',markersize=ms)


ax[0].set_ylabel('$\overline{E}_{11}$')
ax[1].set_ylabel('$\overline{E}_{22}$')
ax[2].set_ylabel('$\overline{E}_{12}$')
ax[3].set_ylabel('$\overline{E}_{\mathrm{eff}}$')

for axs in ax:
    axs.set_xlabel('Global $\overline{E}_{11}$')

ax[3].legend(['Boundaries','Cores','Full map'])

plt.tight_layout()
plt.savefig('mean_b_vs_c.svg')

<>:92: SyntaxWarning: invalid escape sequence '\o'
<>:93: SyntaxWarning: invalid escape sequence '\o'
<>:94: SyntaxWarning: invalid escape sequence '\o'
<>:95: SyntaxWarning: invalid escape sequence '\o'
<>:98: SyntaxWarning: invalid escape sequence '\o'
<>:92: SyntaxWarning: invalid escape sequence '\o'
<>:93: SyntaxWarning: invalid escape sequence '\o'
<>:94: SyntaxWarning: invalid escape sequence '\o'
<>:95: SyntaxWarning: invalid escape sequence '\o'
<>:98: SyntaxWarning: invalid escape sequence '\o'
C:\Users\bepoole\AppData\Local\Temp\ipykernel_24528\2758537633.py:92: SyntaxWarning: invalid escape sequence '\o'
  ax[0].set_ylabel('$\overline{E}_{11}$')
C:\Users\bepoole\AppData\Local\Temp\ipykernel_24528\2758537633.py:93: SyntaxWarning: invalid escape sequence '\o'
  ax[1].set_ylabel('$\overline{E}_{22}$')
C:\Users\bepoole\AppData\Local\Temp\ipykernel_24528\2758537633.py:94: SyntaxWarning: invalid escape sequence '\o'
  ax[2].set_ylabel('$\overline{E}_{12}$')
C:\Users\bepoole\AppDa

In [81]:
plt.rcParams.update({'font.size': 26})
plt.rc('xtick', labelsize=24) 
plt.rc('ytick', labelsize=24) 

fig,ax = plt.subplots(1,4)

fig.set_size_inches(30,7.5)


# fig,ax = plt.subplots(len(d1),1,sharex=True)

# for i in range(0,len(d1)):
#     ax[i].hist(d1[i],bins=100,range=(0,0.1),density=True,alpha=0.5)
#     ax[i].hist(d2[i],bins=100,range=(0,0.1),density=True,alpha=0.5)


# calculate global E11 values 
e11glob = []

for inc, dic_map in exp.iter_over_maps('hrdic'):
    e11glob.append(np.nanmean(dic_map.data['e'][0,0].flatten()))

e11glob = e11glob[:-1]

ccol    = 'forestgreen'
cmark   = 'h'

bcol    = 'darkorange'
bmark   = 'x'

tcol    = 'black'
tmark   = '+'

# e11
d1 = e11gb[:-1]
d2 = e11gc[:-1]
d1mean = [np.nanmedian(x) for x in d1]
d2mean = [np.nanmedian(x) for x in d2]

d3ar = np.hstack((np.asarray(d1),np.asarray(d2)))
d3mean = np.nanmedian(d3ar,1)


ax[0].plot(e11glob,d1mean,marker=bmark,color=bcol,linestyle='None')
ax[0].plot(e11glob,d2mean,marker=cmark,color=ccol,linestyle='None')
ax[0].plot(e11glob,d3mean,marker=tmark,color=tcol,linestyle='None')


# e22 
d1 = e22gb[:-1]
d2 = e22gc[:-1]
d1mean = [np.nanmedian(x) for x in d1]
d2mean = [np.nanmedian(x) for x in d2]

d3ar = np.hstack((np.asarray(d1),np.asarray(d2)))
d3mean = np.nanmedian(d3ar,1)

ax[1].plot(e11glob,d1mean,marker=bmark,color=bcol,linestyle='None')
ax[1].plot(e11glob,d2mean,marker=cmark,color=ccol,linestyle='None')
ax[1].plot(e11glob,d3mean,marker=tmark,color=tcol,linestyle='None')


# e12 
d1 = e12gb[:-1]
d2 = e12gc[:-1]
d1mean = [np.nanmedian(x) for x in d1]
d2mean = [np.nanmedian(x) for x in d2]

d3ar = np.hstack((np.asarray(d1),np.asarray(d2)))
d3mean = np.nanmedian(d3ar,1)

ax[2].plot(e11glob,d1mean,marker=bmark,color=bcol,linestyle='None')
ax[2].plot(e11glob,d2mean,marker=cmark,color=ccol,linestyle='None')
ax[2].plot(e11glob,d3mean,marker=tmark,color=tcol,linestyle='None')


# ems 
d1 = emsgb[:-1]
d2 = emsgc[:-1]
d1mean = [np.nanmedian(x) for x in d1]
d2mean = [np.nanmedian(x) for x in d2]

d3ar = np.hstack((np.asarray(d1),np.asarray(d2)))
d3mean = np.nanmedian(d3ar,1)

ax[3].plot(e11glob,d1mean,marker=bmark,color=bcol,linestyle='None')
ax[3].plot(e11glob,d2mean,marker=cmark,color=ccol,linestyle='None')
ax[3].plot(e11glob,d3mean,marker=tmark,color=tcol,linestyle='None')


ax[0].set_ylabel('$\overline{E}_{11}$')
ax[1].set_ylabel('$\overline{E}_{22}$')
ax[2].set_ylabel('$\overline{E}_{12}$')
ax[3].set_ylabel('$\overline{E}_{\mathrm{eff}}$')

for axs in ax:
    axs.set_xlabel('Global $\overline{E}_{11}$')

ax[3].legend(['Boundaries','Cores','Full map'])

plt.tight_layout()
plt.savefig('median_b_vs_c.tif',dpi=500)

<>:86: SyntaxWarning: invalid escape sequence '\o'
<>:87: SyntaxWarning: invalid escape sequence '\o'
<>:88: SyntaxWarning: invalid escape sequence '\o'
<>:89: SyntaxWarning: invalid escape sequence '\o'
<>:92: SyntaxWarning: invalid escape sequence '\o'
<>:86: SyntaxWarning: invalid escape sequence '\o'
<>:87: SyntaxWarning: invalid escape sequence '\o'
<>:88: SyntaxWarning: invalid escape sequence '\o'
<>:89: SyntaxWarning: invalid escape sequence '\o'
<>:92: SyntaxWarning: invalid escape sequence '\o'
C:\Users\bepoole\AppData\Local\Temp\ipykernel_24528\2657242693.py:86: SyntaxWarning: invalid escape sequence '\o'
  ax[0].set_ylabel('$\overline{E}_{11}$')
C:\Users\bepoole\AppData\Local\Temp\ipykernel_24528\2657242693.py:87: SyntaxWarning: invalid escape sequence '\o'
  ax[1].set_ylabel('$\overline{E}_{22}$')
C:\Users\bepoole\AppData\Local\Temp\ipykernel_24528\2657242693.py:88: SyntaxWarning: invalid escape sequence '\o'
  ax[2].set_ylabel('$\overline{E}_{12}$')
C:\Users\bepoole\AppDa

In [102]:
# extreme strain values 

plt.rcParams.update({'font.size': 26})
plt.rc('xtick', labelsize=24) 
plt.rc('ytick', labelsize=24) 

fig,ax = plt.subplots(1,4)

fig.set_size_inches(30,7.5)


# calculate global E11 values 
e11glob = []

for inc, dic_map in exp.iter_over_maps('hrdic'):
    e11glob.append(np.nanmean(dic_map.data['e'][0,0].flatten()))

e11glob = e11glob[:-1]

pc = 95

# e11
d1 = e11gb[:-1]
d2 = e11gc[:-1]
d1pmax = [np.nanpercentile(abs(x),pc) for x in d1]
d2pmax = [np.nanpercentile(abs(x),pc)  for x in d2]

d3ar = np.hstack((np.asarray(d1),np.asarray(d2)))
d3pmax = np.nanpercentile(abs(d3ar),pc,axis=1)

ax[0].plot(e11glob,d1pmax,marker=bmark,color=bcol,linestyle='None',markersize=ms)
ax[0].plot(e11glob,d2pmax,marker=cmark,color=ccol,linestyle='None',markersize=ms)
ax[0].plot(e11glob,d3pmax,marker=tmark,color=tcol,linestyle='None',markersize=ms)


# e22 
d1 = e22gb[:-1]
d2 = e22gc[:-1]
d1pmax = [np.nanpercentile(abs(x),pc) for x in d1]
d2pmax = [np.nanpercentile(abs(x),pc)  for x in d2]

d3ar = np.hstack((np.asarray(d1),np.asarray(d2)))
d3pmax = np.nanpercentile(abs(d3ar),pc,axis=1)

ax[1].plot(e11glob,d1pmax,marker=bmark,color=bcol,linestyle='None',markersize=ms)
ax[1].plot(e11glob,d2pmax,marker=cmark,color=ccol,linestyle='None',markersize=ms)
ax[1].plot(e11glob,d3pmax,marker=tmark,color=tcol,linestyle='None',markersize=ms)

# e12 
d1 = e12gb[:-1]
d2 = e12gc[:-1]
d1pmax = [np.nanpercentile(abs(x),pc) for x in d1]
d2pmax = [np.nanpercentile(abs(x),pc)  for x in d2]

d3ar = np.hstack((np.asarray(d1),np.asarray(d2)))
d3pmax = np.nanpercentile(abs(d3ar),pc,axis=1)

ax[2].plot(e11glob,d1pmax,marker=bmark,color=bcol,linestyle='None',markersize=ms)
ax[2].plot(e11glob,d2pmax,marker=cmark,color=ccol,linestyle='None',markersize=ms)
ax[2].plot(e11glob,d3pmax,marker=tmark,color=tcol,linestyle='None',markersize=ms)


# ems 
d1 = emsgb[:-1]
d2 = emsgc[:-1]
d1pmax = [np.nanpercentile(abs(x),pc) for x in d1]
d2pmax = [np.nanpercentile(abs(x),pc)  for x in d2]

d3ar = np.hstack((np.asarray(d1),np.asarray(d2)))
d3pmax = np.nanpercentile(abs(d3ar),pc,axis=1)

ax[3].plot(e11glob,d1pmax,marker=bmark,color=bcol,linestyle='None',markersize=ms)
ax[3].plot(e11glob,d2pmax,marker=cmark,color=ccol,linestyle='None',markersize=ms)
ax[3].plot(e11glob,d3pmax,marker=tmark,color=tcol,linestyle='None',markersize=ms)

ax[0].set_ylabel('$pc_{95}(|E_{11}|)$')
ax[1].set_ylabel('$pc_{95}(|E_{22}|)$')
ax[2].set_ylabel('$pc_{95}(|E_{12}|)$')
ax[3].set_ylabel('$pc_{95}(E_{\mathrm{eff}})$')

ax[0].set_xlabel('Global $\overline{E}_{11}$')
ax[1].set_xlabel('Global $\overline{E}_{11}$')
ax[2].set_xlabel('Global $\overline{E}_{11}$')
ax[3].set_xlabel('Global $\overline{E}_{11}$')



ax[3].legend(['Boundaries','Cores','Full map'])

plt.tight_layout()
plt.savefig('pc99_b_vs_c.svg')

<>:79: SyntaxWarning: invalid escape sequence '\m'
<>:81: SyntaxWarning: invalid escape sequence '\o'
<>:82: SyntaxWarning: invalid escape sequence '\o'
<>:83: SyntaxWarning: invalid escape sequence '\o'
<>:84: SyntaxWarning: invalid escape sequence '\o'
<>:79: SyntaxWarning: invalid escape sequence '\m'
<>:81: SyntaxWarning: invalid escape sequence '\o'
<>:82: SyntaxWarning: invalid escape sequence '\o'
<>:83: SyntaxWarning: invalid escape sequence '\o'
<>:84: SyntaxWarning: invalid escape sequence '\o'
C:\Users\bepoole\AppData\Local\Temp\ipykernel_24528\218803430.py:79: SyntaxWarning: invalid escape sequence '\m'
  ax[3].set_ylabel('$pc_{95}(E_{\mathrm{eff}})$')
C:\Users\bepoole\AppData\Local\Temp\ipykernel_24528\218803430.py:81: SyntaxWarning: invalid escape sequence '\o'
  ax[0].set_xlabel('Global $\overline{E}_{11}$')
C:\Users\bepoole\AppData\Local\Temp\ipykernel_24528\218803430.py:82: SyntaxWarning: invalid escape sequence '\o'
  ax[1].set_xlabel('Global $\overline{E}_{11}$')
C:\

In [12]:
#time evolving histograms 

savedir = './histogram_evo'
dpi = 500

Path(savedir).mkdir(parents=True, exist_ok=True)


# calculate global E11 values 
e11glob = []

for inc, dic_map in exp.iter_over_maps('hrdic'):
    e11glob.append(np.nanmean(dic_map.data['e'][0,0].flatten()))



for inc, dic_map in exp.iter_over_maps('hrdic'):
    # plotter to make stack of plots with stress strain curve and moving pointer 

    fig = plt.figure()
    gs = GridSpec(4,3,figure=fig)
    fig.set_size_inches(10,5)


    # curves 
    # top left 
    ax0 = fig.add_subplot(gs[1:3,0])

    x = 'Eng Strain'
    y = 'Eng Stress / MPa'

    ax0.plot(rigdat[x],rigdat[y],'k',linewidth = 1)
    ax0.plot(rigdatimage[x],rigdatimage[y],'.',markerfacecolor='orange',markeredgecolor='green',markersize=15,alpha=0.9)
    
    # bodge because unload image isn't in rig data experiment file 
    ax0.plot(rigdat.iloc[-1][x],rigdat.iloc[-1][y],'.',markerfacecolor='orange',markeredgecolor='green',markersize=15,alpha=0.9)
    try:
        ax0.plot(rigdatimage.iloc[inc+1][x],rigdatimage.iloc[inc+1][y],'.',markerfacecolor=None,markeredgecolor='green',markersize=20,alpha=0.9)
    except: 
        ax0.plot(rigdat.iloc[-1][x],rigdat.iloc[-1][y],'.',markerfacecolor=None,markeredgecolor='green',markersize=20,alpha=0.9)
    
    ax0.set_ylabel(y)
    ax0.set_xlabel(x)

    
    # histograms 
    ymax = 50

    # e11 
    ax1 = fig.add_subplot(gs[0,1:])
    ax1.hist(e11gb[inc],range=(-0.1,0.1),bins=200,density=True,alpha=0.5,color=bcol)
    # ax1.axvline(np.nanpercentile(e11gb[inc],50),color=bcol)
    
    ax1.hist(e11gc[inc],range=(-0.1,0.1),bins=200,density=True,alpha=0.5,color=ccol)
    # ax1.axvline(np.nanpercentile(e11gc[inc],50),color=ccol)

    ax1.set_ylim(0,ymax)
    
    # e22 
    ax2 = fig.add_subplot(gs[1,1:])
    ax2.hist(e22gb[inc],range=(-0.1,0.1),bins=200,density=True,alpha=0.5,color=bcol)
    # ax2.axvline(np.nanpercentile(e22gb[inc],50),color=bcol)
    
    ax2.hist(e22gc[inc],range=(-0.1,0.1),bins=200,density=True,alpha=0.5,color=ccol)
    # ax2.axvline(np.nanpercentile(e22gc[inc],50),color=ccol)

    ax2.set_ylim(0,ymax)

    # e12 
    ax3 = fig.add_subplot(gs[2,1:])
    ax3.hist(e12gb[inc],range=(-0.1,0.1),bins=200,density=True,alpha=0.5,color=bcol)
    # ax3.axvline(np.nanpercentile(e12gb[inc],50),color=bcol)
    
    ax3.hist(e12gc[inc],range=(-0.1,0.1),bins=200,density=True,alpha=0.5,color=ccol)
    # ax3.axvline(np.nanpercentile(e12gc[inc],50),color=ccol)

    ax3.set_ylim(0,ymax)

    # ems 
    ax4 = fig.add_subplot(gs[3,1:])
    ax4.hist(emsgb[inc],range=(0,0.1),bins=200,density=True,alpha=0.5,color=bcol)
    # ax4.axvline(np.nanpercentile(emsgb[inc],50),color=bcol)
    
    ax4.hist(emsgc[inc],range=(0,0.1),bins=200,density=True,alpha=0.5,color=ccol)
    # ax4.axvline(np.nanpercentile(emsgc[inc],50),color=ccol)

    ax4.set_ylim(0,ymax)

    plt.tight_layout()

    # save the figures 
    plt.savefig(savedir + '/increment_'+str(inc)+'.tif',dpi=dpi)
    plt.close()


In [23]:
# plots for gifs with strain progression

savedir = './strain_evo_with_gbs'
dpi = 500

Path(savedir).mkdir(parents=True, exist_ok=True)


for inc, dic_map in exp.iter_over_maps('hrdic'):
    # plotter to make stack of plots with stress strain curve and moving pointer 

    fig = plt.figure(layout="constrained")
    fig.set_size_inches(24,12)
    gs = GridSpec(2,2,figure=fig)

    # curves 
    # top left 
    ax0 = fig.add_subplot(gs[0,0])

    x = 'Eng Strain'
    y = 'Eng Stress / MPa'

    ax0.plot(rigdat[x],rigdat[y],'k',linewidth = 1)
    ax0.plot(rigdatimage[x],rigdatimage[y],'.',markerfacecolor='orange',markeredgecolor='green',markersize=15,alpha=0.9)
    
    # bodge because unload image isn't in rig data experiment file 
    ax0.plot(rigdat.iloc[-1][x],rigdat.iloc[-1][y],'.',markerfacecolor='orange',markeredgecolor='green',markersize=15,alpha=0.9)
    try:
        ax0.plot(rigdatimage.iloc[inc+1][x],rigdatimage.iloc[inc+1][y],'.',markerfacecolor=None,markeredgecolor='green',markersize=20,alpha=0.9)
    except: 
        ax0.plot(rigdat.iloc[-1][x],rigdat.iloc[-1][y],'.',markerfacecolor=None,markeredgecolor='green',markersize=20,alpha=0.9)
    
    ax0.set_ylabel(y)
    ax0.set_xlabel(x)


    # curves 
    # bottom left 
    ax1 = fig.add_subplot(gs[1,0])
    x = 'Elapsed Time / s'
    y = 'Force / N'

    ax1.plot(rigdat[x]/(60**2),rigdat[y],'k',linewidth = 1)
    ax1.plot(rigdatimage[x]/(60**2),rigdatimage[y],'.',markerfacecolor='orange',markeredgecolor='green',markersize=15,alpha=0.9)
    
    # bodge because unload image isn't in rig data experiment file 
    ax1.plot(rigdat.iloc[-1][x]/(60**2),rigdat.iloc[-1][y],'.',markerfacecolor='orange',markeredgecolor='green',markersize=15,alpha=0.9)
    try:
        ax1.plot(rigdatimage.iloc[inc+1][x]/(60**2),rigdatimage.iloc[inc+1][y],'.',markerfacecolor=None,markeredgecolor='green',markersize=20,alpha=0.9)
    except: 
        ax1.plot(rigdat.iloc[-1][x]/(60**2),rigdat.iloc[-1][y],'.',markerfacecolor=None,markeredgecolor='green',markersize=20,alpha=0.9)
    
    ax1.set_ylabel(y)
    ax1.set_xlabel('Time / h')


    ax2 = fig.add_subplot(gs[:,1])

    dic_map.plot_map(
        'max_shear', vmin=0, vmax=0.10, 
        plot_scale_bar=True, plot_gbs='line',
        fig=fig,ax=ax2
    )

    # save the figures 
    plt.savefig(savedir + '/increment_'+str(inc)+'.tif',dpi=dpi)
    plt.close()


In [6]:
# plots for gifs with strain progression

savedir = './strain_evo_no_graph'
dpi = 500

Path(savedir).mkdir(parents=True, exist_ok=True)


for inc, dic_map in exp.iter_over_maps('hrdic'):
    # plotter to make stack of plots with stress strain curve and moving pointer 

    fig,ax = plt.subplots()
    fig.set_size_inches(12,12)

    dic_map.plot_map(
        'max_shear', vmin=0, vmax=0.10, 
        plot_scale_bar=False, plot_gbs=False,
        fig=fig,ax=ax
    )

    # save the figures 
    plt.savefig(savedir + '/increment_'+str(inc)+'.tif',dpi=dpi)
    plt.close()

In [4]:
# plots for gifs with strain progression

savedir = './strain_evo_mega'
dpi = 500

Path(savedir).mkdir(parents=True, exist_ok=True)


for inc, dic_map in exp.iter_over_maps('hrdic'):
    # plotter to make stack of plots with stress strain curve and moving pointer 

    fig = plt.figure(layout="constrained")
    fig.set_size_inches(24,12)
    gs = GridSpec(2,2,figure=fig)

    # curves 
    # top left 
    ax0 = fig.add_subplot(gs[0,0])

    x = 'Eng Strain'
    y = 'Eng Stress / MPa'

    ax0.plot(rigdat[x],rigdat[y],'k',linewidth = 1)
    ax0.plot(rigdatimage[x],rigdatimage[y],'.',markerfacecolor='orange',markeredgecolor='green',markersize=15,alpha=0.9)
    
    # bodge because unload image isn't in rig data experiment file 
    ax0.plot(rigdat.iloc[-1][x],rigdat.iloc[-1][y],'.',markerfacecolor='orange',markeredgecolor='green',markersize=15,alpha=0.9)
    try:
        ax0.plot(rigdatimage.iloc[inc+1][x],rigdatimage.iloc[inc+1][y],'.',markerfacecolor=None,markeredgecolor='green',markersize=20,alpha=0.9)
    except: 
        ax0.plot(rigdat.iloc[-1][x],rigdat.iloc[-1][y],'.',markerfacecolor=None,markeredgecolor='green',markersize=20,alpha=0.9)
    
    ax0.set_ylabel(y)
    ax0.set_xlabel(x)


    # curves 
    # bottom left 
    ax1 = fig.add_subplot(gs[1,0])
    x = 'Elapsed Time / s'
    y = 'Force / N'

    ax1.plot(rigdat[x]/(60**2),rigdat[y],'k',linewidth = 1)
    ax1.plot(rigdatimage[x]/(60**2),rigdatimage[y],'.',markerfacecolor='orange',markeredgecolor='green',markersize=15,alpha=0.9)
    
    # bodge because unload image isn't in rig data experiment file 
    ax1.plot(rigdat.iloc[-1][x]/(60**2),rigdat.iloc[-1][y],'.',markerfacecolor='orange',markeredgecolor='green',markersize=15,alpha=0.9)
    try:
        ax1.plot(rigdatimage.iloc[inc+1][x]/(60**2),rigdatimage.iloc[inc+1][y],'.',markerfacecolor=None,markeredgecolor='green',markersize=20,alpha=0.9)
    except: 
        ax1.plot(rigdat.iloc[-1][x]/(60**2),rigdat.iloc[-1][y],'.',markerfacecolor=None,markeredgecolor='green',markersize=20,alpha=0.9)
    
    ax1.set_ylabel(y)
    ax1.set_xlabel('Time / h')


    ax2 = fig.add_subplot(gs[:,1])

    dic_map.plot_map(
        'max_shear', vmin=0, vmax=0.10, 
        plot_scale_bar=False, plot_gbs=False,
        fig=fig,ax=ax2
    )

    # save the figures 
    plt.savefig(savedir + '/increment_'+str(inc)+'.tif',dpi=dpi)
    plt.close()


In [11]:
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

In [7]:
fig,ax = plt.subplots()
fig.set_size_inches(20,20)
plt.rcParams['svg.fonttype'] = 'none'

dic_map = exp.increments[-1].maps['hrdic']
dic_map.plot_map(
    'max_shear', vmin=0, vmax=0.10, 
    plot_scale_bar=True, plot_gbs=False,
    fig=fig,ax=ax,cmap='turbo'
)

plt.tight_layout()

In [17]:
plt.savefig('expo_no_gbs.pdf',dpi=1000)

In [11]:
fig,ax = plt.subplots()
fig.set_size_inches(6,6)
ebsd_map = exp.increments[0].maps['ebsd']
ebsd_map.plot_ipf_map([1,0,0],plot_scale_bar=True,plot_gbs=False,fig=fig,ax=ax)
plt.tight_layout()

plt.savefig('ebsd_pre.tif',dpi=500)

In [10]:
#remove outliers 

emsgbclean = []
emsgcclean = []

emstot = []

ccol    = 'forestgreen'
cmark   = 'h'

bcol    = 'darkorange'
bmark   = 'x'

tcol    = 'black'
tmark   = '+'

pc = 99.9

for i in range(0,len(emsgb)):
    xb = copy.copy(emsgb[i])
    xb[xb>np.nanpercentile(xb,pc)] = np.nan

    xb = xb[~np.isnan(xb)]

    emsgbclean.append(xb)


    xc = copy.copy(emsgc[i])
    xc[xc>np.nanpercentile(xc,pc)] = np.nan

    xc = xc[~np.isnan(xc)]

    emsgcclean.append(xc)

    x =np.concatenate([xb, xc])

    emstot.append(x)

# calculate global E11 values 
e11glob = []

for inc, dic_map in exp.iter_over_maps('hrdic'):
    e11glob.append(np.nanmean(dic_map.data['e'][0,0].flatten()))


In [16]:


rb = []
rc =  []


for i in range(0,len(emstot)):
    rb.append(np.sum(emsgbclean[i])/np.sum(emstot[i]))
    rc.append(np.sum(emsgcclean[i])/np.sum(emstot[i]))


fig,ax = plt.subplots()

for i in range(0,len(emstot)):
    ax.plot(e11glob[i],emstot[i].mean(),'k+')
    ax.plot(e11glob[i],emsgbclean[i].mean(),marker=bmark,color=bcol,linestyle='None')
    ax.plot(e11glob[i],emsgcclean[i].mean(),marker=cmark,color=ccol,linestyle='None')

ax.set_xlabel('Global $\overline{E}_{11}$')
ax.set_ylabel('$\overline{E}_{\mathrm{eff}}$')    
ax.legend(['Entire map','Boundaries','Cores'])

plt.tight_layout()

# plt.savefig('mean_strains_b_vs_c_tot.tif',dpi=500)




plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams.update({'font.size': 26})
plt.rc('xtick', labelsize=24) 
plt.rc('ytick', labelsize=24) 

fig,ax = plt.subplots(2,1,sharex=True)
fig.set_size_inches(20,20)

ax[1].plot(e11glob,rb,marker=bmark,color=bcol,linestyle='None',markersize=20)
ax[1].axhline(len(emsgbclean[0])/len(emstot[0]),color=bcol,linestyle='--')
# ax[1].set_ylim([0.1,0.2])
ax[1].set_xlabel('Global $\overline{E}_{11}$')
ax[1].set_ylabel('$f_{B}$')

ax[0].plot(e11glob,rc,marker=bmark,color=ccol,linestyle='None',markersize=20)
ax[0].axhline(len(emsgcclean[0])/len(emstot[0]),color=ccol,linestyle='--')
# ax[0].set_ylim([0.8,0.9])
ax[0].set_ylabel('$f_{C}$')

plt.tight_layout()

plt.savefig('strain_fractions.svg')#,dpi=500)

<>:17: SyntaxWarning: invalid escape sequence '\o'
<>:18: SyntaxWarning: invalid escape sequence '\o'
<>:39: SyntaxWarning: invalid escape sequence '\o'
<>:17: SyntaxWarning: invalid escape sequence '\o'
<>:18: SyntaxWarning: invalid escape sequence '\o'
<>:39: SyntaxWarning: invalid escape sequence '\o'
C:\Users\bepoole\AppData\Local\Temp\ipykernel_14448\1668532764.py:17: SyntaxWarning: invalid escape sequence '\o'
  ax.set_xlabel('Global $\overline{E}_{11}$')
C:\Users\bepoole\AppData\Local\Temp\ipykernel_14448\1668532764.py:18: SyntaxWarning: invalid escape sequence '\o'
  ax.set_ylabel('$\overline{E}_{\mathrm{eff}}$')
C:\Users\bepoole\AppData\Local\Temp\ipykernel_14448\1668532764.py:39: SyntaxWarning: invalid escape sequence '\o'
  ax[1].set_xlabel('Global $\overline{E}_{11}$')


Bad stuff below here 

In [ ]:
rootdir = 'Post_HREBSD/'
e11 = pd.read_excel(rootdir + 'e11.xlsx')
e12 = pd.read_excel(rootdir + 'e12.xlsx')
e22 = pd.read_excel(rootdir + 'e22.xlsx')

gndt = pd.read_excel(rootdir + 'gndtot.xlsx')
gnde = pd.read_excel(rootdir + 'gndedge.xlsx')
gnds = pd.read_excel(rootdir + 'gndscrew.xlsx')

s11 = pd.read_excel(rootdir + 's11.xlsx')
s12 = pd.read_excel(rootdir + 's12.xlsx')
s22 = pd.read_excel(rootdir + 's22.xlsx')

w12 = pd.read_excel(rootdir + 'w12.xlsx')
w23 = pd.read_excel(rootdir + 'w23.xlsx')
w31 = pd.read_excel(rootdir + 'w31.xlsx')
ws = pd.read_excel(rootdir + 'wscalar.xlsx')


In [ ]:
plt.figure()
plt.imshow(np.rot90(np.rot90(e11)))

In [ ]:
dic_map = exp.increments[0].maps['hrdic']
dic_map.set_crop(left=100,right=100,top=100,bottom=100)

In [ ]:
dic_map = exp.increments[0].maps['hrdic']
dic_map.plot_map('max_shear',vmin=0,vmax=0.005,plot_scale_bar=True
)

In [90]:
import matplotlib.gridspec as gridspec

plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams.update({'font.size': 26})
plt.rc('xtick', labelsize=24) 
plt.rc('ytick', labelsize=24) 

fig = plt.figure()


# sizes
fig.set_size_inches(12,6)
spec = gridspec.GridSpec(6,1,figure=fig)

bins = 80

# datasource = dic_map.data['e'][0,0]
datasource = dic_map.data['max_shear']

ax0 = fig.add_subplot(spec[1:])
ax0.hist(datasource.flatten(),range=(0,0.01),bins=bins,density=True,color='darkgreen')
ax0.set_xlabel('$E_{\mathrm{eff}}$')
# ax0.set_xlabel('$E_{eff}$')
ax0.set_ylabel('Prob. density')

# ax0.axvline(0,color='k',linewidth=1)

ax1 = fig.add_subplot(spec[0:1],sharex=ax0)
ax1.boxplot(datasource.flatten(),vert=False,showfliers=False,widths=10)
ax1.axis('off')





plt.tight_layout()
plt.savefig('Ems_noise.svg')#,dpi=300)

<>:22: SyntaxWarning: invalid escape sequence '\m'
<>:22: SyntaxWarning: invalid escape sequence '\m'
C:\Users\bepoole\AppData\Local\Temp\ipykernel_27276\2049416606.py:22: SyntaxWarning: invalid escape sequence '\m'
  ax0.set_xlabel('$E_{\mathrm{eff}}$')
